# Simulation experiment using noisy data

Run entire simulation experiment multiple times to generate confidence interval.  The simulation experiment can be found in ```functions/pipeline.py```

In [1]:
%load_ext autoreload
%autoreload 2

from joblib import Parallel, delayed
import multiprocessing
import sys
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

sys.path.append("../../")
from functions import pipelines

from numpy.random import seed
randomState = 123
seed(randomState)

Using TensorFlow backend.


In [2]:
# Parameters
dataset_name = "Human_analysis"
analysis_name = 'Human_experiment_lvl_sim'
NN_architecture = 'NN_2500_30'
file_prefix = "Partition"
num_simulated_experiments = 50
lst_num_partitions = [1, 2, 3, 5, 10, 20,
                    30, 50]

corrected = False
use_pca = True
num_PCs = 10

iterations = range(5) 
num_cores = 5

In [3]:
# Input
base_dir = os.path.abspath(
      os.path.join(
          os.getcwd(), "../.."))

normalized_data_file = os.path.join(
    base_dir,
    dataset_name,    
    "data",
    "input",
    "recount2_gene_normalized_data.tsv.xz")

experiment_ids_file = os.path.join(
    base_dir,
    dataset_name,
    "data",
    "metadata",
    "recount2_experiment_ids.txt")

In [4]:
# Output files
similarity_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Human_experiment_lvl_sim_similarity_uncorrected.pickle")

ci_uncorrected_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Human_experiment_lvl_sim_ci_uncorrected.pickle")

similarity_permuted_file = os.path.join(
    base_dir,
    "results",
    "saved_variables",
    "Human_experiment_lvl_sim_permuted")

In [5]:
# Run multiple simulations - uncorrected
results = Parallel(n_jobs=num_cores, verbose=100)(
    delayed(
        pipelines.experiment_level_simulation_uncorrected)(i,
                                                           NN_architecture,
                                                           dataset_name,
                                                           analysis_name,
                                                           num_simulated_experiments,
                                                           lst_num_partitions,
                                                           corrected,
                                                           use_pca,
                                                           num_PCs,
                                                           file_prefix,
                                                           normalized_data_file,
                                                           experiment_ids_file) for i in iterations)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed: 15.7min
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 18.5min remaining: 27.8min
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 23.5min remaining: 15.7min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 30.8min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 30.8min finished


In [6]:
base_dir

'/home/alexandra/Documents/Repos/Batch_effects_simulation'

In [7]:
# permuted score
permuted_score = results[0][0]

In [8]:
# Concatenate output dataframes
all_svcca_scores = pd.DataFrame()

for i in iterations:
    all_svcca_scores = pd.concat([all_svcca_scores, results[i][1]], axis=1)

all_svcca_scores

,score,score,score,score,score
number of partitions,,,,,
1,0.999968,0.999973,0.999980,0.999962,0.999972
2,0.914205,0.947893,0.908161,0.930284,0.906635
10,0.824886,0.738349,0.803153,0.795071,0.760857
20,0.852677,0.841464,0.844727,0.858762,0.821125


In [9]:
# Get median for each row (number of experiments)
mean_scores = all_svcca_scores.mean(axis=1).to_frame()
mean_scores.columns = ['score']
mean_scores

,score
number of partitions,
1,0.999971
2,0.921436
10,0.784463
20,0.843751


In [10]:
# Get standard dev for each row (number of experiments)
import math
std_scores = (all_svcca_scores.std(axis=1)/math.sqrt(10)).to_frame()
std_scores.columns = ['score']
std_scores

,score
number of partitions,
1,0.000002
2,0.005536
10,0.010931
20,0.004536


In [11]:
# Get confidence interval for each row (number of experiments)
err = std_scores*1.96

In [12]:
# Get boundaries of confidence interval
ymax = mean_scores + err
ymin = mean_scores - err

ci = pd.concat([ymin, ymax], axis=1)
ci.columns = ['ymin', 'ymax']
ci

,ymin,ymax
number of partitions,,
1,0.999967,0.999975
2,0.910585,0.932286
10,0.763038,0.805888
20,0.834860,0.852642


In [13]:
mean_scores

,score
number of partitions,
1,0.999971
2,0.921436
10,0.784463
20,0.843751


In [14]:
# Pickle dataframe of mean scores scores for first run, interval
mean_scores.to_pickle(similarity_uncorrected_file)
ci.to_pickle(ci_uncorrected_file)
np.save(similarity_permuted_file, permuted_score)